# The Template pattern
is usefull for removing duplicate code.The comon steps are implemented in a base class and a distinct steps are overridden in subclasses.


In [1]:
import sqlite3

def test_setup(db_name: str = "sales.db") -> sqlite3.Connection:
    conn = sqlite3.connect(db_name)
    
    conn.execute(
        """
        CREATE TABLE IF NOT EXISTS Sales (
            salesperson text,
            amt currency,
            year integer,
            model text,
            new boolean
        )
        """
    )
    conn.execute(
        """
        DELETE FROM Sales
        """
    )
    conn.execute(
        """
        INSERT INTO Sales
        VALUES('Tim', 16000, 2010, 'Honda Fit', 'true')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales
        VALUES('Tim', 9000, 2006, 'Ford Focus', 'false')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales
        VALUES('Hannah', 8000, 2004, 'Dodge Neon', 'false')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales
        VALUES('Hannah', 28000, 2009, 'Ford Mustang', 'true')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales
        VALUES('Hannah', 50000, 2010, 'Lincoln Navigator', 'true')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales
        VALUES('JAson', 20000, 2008, 'Toyota Prius', 'false')
        """
    )    
    conn.commit()
    return conn


## Base class

In [14]:
from __future__ import annotations
from typing import ContextManager, TextIO, cast
import contextlib
import csv
import sys

class QueryTemplate:
    def __init__(self, db_name: str = "sales.db") -> None:
        self.db_name = db_name
        self.conn: sqlite3.Connection
        self.results: list[tuple[str, ...]]
        self.query: str
        self.header: list[str]
        
    def connect(self) -> None:
        self.conn = sqlite3.connect(self.db_name)
    
    def construct_query(self) -> None:
        raise NotImplementedError("construct_query not implemented")
    
    def do_query(self) -> None:
        results = self.conn.execute(self.query)
        self.results = results.fetchall()
    
    def output_context(self) -> ContextManager[TextIO]:
        self.target_file = sys.stdout
        return cast(ContextManager[TextIO], contextlib.nullcontext())
    
    def output_results(self) -> None:
        writer = csv.writer(self.target_file)
        writer.writerow(self.header)
        writer.writerows(self.results)
    
    def process_format(self) -> None:
        self.connect()
        self.construct_query()
        self.do_query()
        with self.output_context():
            self.output_results()

In [26]:
import datetime
from pathlib import Path

class NewVehiclesQuery(QueryTemplate):
    def construct_query(self) -> None:
        self.query = """
            SELECT * FROM Sales WHERE new='true'
        """
        self.header = ["salesperson", "amt", "year", "model", "new"]
        

class SalesGrossQuery(QueryTemplate):
    def construct_query(self) -> None:
        self.query = """
            SELECT salesperson, sum(amt) FROM Sales GROUP BY salesperson
        """
        self.header = ["salesperson", "total sales"]
        
    def output_context(self) -> ContextManager[TextIO]:
        today = datetime.date.today()
        filepath = Path(f"gross_sales_{today:%Y%m%d}.csv")
        self.target_file = filepath.open("w")
        return self.target_file

In [27]:
def main() -> None:
    test_setup()
    
    task_1 = NewVehiclesQuery()
    task_1.process_format()
    
    task_2 = SalesGrossQuery()
    task_2.process_format()
    
if __name__ == "__main__":
    main()

salesperson,amt,year,model,new
Tim,16000,2010,Honda Fit,true
Hannah,28000,2009,Ford Mustang,true
Hannah,50000,2010,Lincoln Navigator,true
